# Database

## Set up a PostgreSQL database with the provided tables.

### Migrations

#### Alembic

Alembic is a lightweight database migration tool for SQLAlchemy, a Python SQL toolkit and Object-Relational Mapper (ORM). It provides a convenient way to manage changes to your database schema over time.

##### Key Features:

- **Schema Version Control:** Alembic enables tracking schema changes in databases through versioned revisions. Each revision represents a change in the database schema.

- **Automatic Migration Generation:** Alembic can automatically generate migration scripts based on the differences between SQLAlchemy models and the current state of the database.

- **Custom Migrations:** Developers can create custom migrations using Python, providing flexibility to handle complex migration cases.

- **Multiple Database Support:** Alembic is compatible with a variety of database management systems (DBMS) supported by SQLAlchemy, such as PostgreSQL, MySQL, SQLite, among others.

##### Basic Usage:

1. **Initial Setup:** Initialize the Alembic environment and configure the `alembic.ini` file with the database connection information.

2. **Creation of Revisions:** Use the `alembic revision -m "message"` command to create a new revision. This will create a new Python file in the revisions folder.

3. **Database Update:** Apply pending revisions to the database using the `alembic upgrade head` command.

4. **Undo Changes:** If needed, it's possible to revert applied changes using the `alembic downgrade -1` command, where `-1` represents the number of revisions to be reverted.


#### Migration File used

In [3]:
!cat migrations/versions/397c930c2bfd_create_tables.py

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
"""Create Tables

Revision ID: 397c930c2bfd
Revises:
Create Date: 2024-02-10 14:06:45.066698
"""
from typing import Sequence, Union

from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision: str = '397c930c2bfd'
down_revision: Union[str, None] = None
branch_labels: Union[str, Sequence[str], None] = None
depends_on: Union[str, Sequence[str], None] = None


def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.create_table(
        'clients',
        sa.Column('user_id', sa.Integer(), nullable=False, autoincrement=True),
        sa.Column('created_at', sa.DateTime(), nullable=True),
        sa.Column('status', sa.String(), nullable=True),
        sa.Column('batch', sa.Integer(), nullable=True),
        sa.Column('credit_limit', sa.Integer(), nullable=Tru

In [4]:
!cat migrations/versions/3168150c1208_insert_clients_data.py

"""insert clients data

Revision ID: 3168150c1208
Revises: 397c930c2bfd
Create Date: 2024-02-10 14:06:56.907964
"""
from typing import Sequence, Union
import os
import csv

from alembic import op
from sqlalchemy import text

# revision identifiers, used by Alembic.
revision: str = '3168150c1208'
down_revision: Union[str, None] = '397c930c2bfd'
branch_labels: Union[str, Sequence[str], None] = None
depends_on: Union[str, Sequence[str], None] = None


def upgrade() -> None:
    dirname = os.getcwd()
    filename = os.path.join(dirname, 'migrations/data/clients.csv')
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        columns = ','.join(header)
        conn = op.get_bind().engine.raw_connection()
        cursor = conn.cursor()
        cmd = f'COPY clients({columns}) FROM STDIN WITH (FORMAT CSV, HEADER FALSE)'
        cursor.copy_expert(cmd, f)
        conn.commit()
    op.execute(
        "SELECT setval('clients_user_id_seq', (SELECT MAX(

In [5]:
!cat migrations/versions/0d82ccbd5dee_insert_loans_data.py

"""insert loanss data

Revision ID: 0d82ccbd5dee
Revises: 3168150c1208
Create Date: 2024-02-10 14:07:04.679731
"""
from typing import Sequence, Union
import csv
import os

from alembic import op

# revision identifiers, used by Alembic.
revision: str = '0d82ccbd5dee'
down_revision: Union[str, None] = '3168150c1208'
branch_labels: Union[str, Sequence[str], None] = None
depends_on: Union[str, Sequence[str], None] = None


def upgrade() -> None:
    dirname = os.getcwd()
    filename = os.path.join(dirname, 'migrations/data/loans.csv')
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        columns = ','.join(header)
        conn = op.get_bind().engine.raw_connection()
        cursor = conn.cursor()
        cmd = (
            f'COPY loans({columns}) FROM STDIN WITH (FORMAT CSV, HEADER FALSE)'
        )
        cursor.copy_expert(cmd, f)
        conn.commit()
    op.execute(
        "SELECT setval('loans_loan_id_seq', (SELECT MAX(loan_id) FR

### Docker Compose

##### Docker Compose YML file

In [6]:
!cat docker-compose.yml

version: '3.8'

services:
  db:
    image: postgres:latest
    restart: always
    environment:
      POSTGRES_DB: ${POSTGRES_DB}
      POSTGRES_USER: ${POSTGRES_USER}
      POSTGRES_PASSWORD: ${POSTGRES_PASSWORD}
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data
    network_mode: host

  pgadmin:
    image: dpage/pgadmin4
    container_name: pgadmin4_container
    restart: always
    ports:
      - "8889:80"
    depends_on:
      - db
    environment:
      PGADMIN_DEFAULT_EMAIL: admin@admin.com
      PGADMIN_DEFAULT_PASSWORD: admin
    network_mode: host
      
  app:
    image: app
    build:
      context: .
      dockerfile: Dockerfile
    ports:
      - "8000:8000"
    depends_on:
      - db
    network_mode: host

volumes:
  postgres_data:


##### Docker running

In [1]:
!docker compose up -d

[+] Running 3/0
 ✔ Container cloudwalk-db-1      Running                                   0.0s 
 ✔ Container cloudwalk-app-1     Running                                   0.0s 
 ✔ Container pgadmin4_container  Running                                   0.0s 


#### db

The `db` service is configured using the `postgres:latest` image. It will be used as the primary database to store data related to loan operations. Here's a detailed explanation of the settings:

- **image**: Specifies the PostgreSQL image to be used.
- **restart**: Sets the container's restart policy to "always", ensuring that the container is automatically restarted in case of failure.
- **environment**: Defines the necessary environment variables to configure the database, including the database name, username, and password.
- **ports**: Maps the PostgreSQL default port (5432) from the container to port 5432 on the host, allowing external access to the database.
- **volumes**: Defines a volume to persist PostgreSQL data, storing it on the host.
- **network_mode**: Sets the network mode to "host" to simplify communication between containers.

#### pgAdmin

The `pgadmin` service is configured using the `dpage/pgadmin4` image. It provides a web interface for managing the PostgreSQL database. Here's a detailed explanation of the settings:

- **image**: Specifies the pgAdmin image to be used.
- **container_name**: Sets the container name to "pgadmin4_container" for easy reference.
- **restart**: Sets the container's restart policy to "always", ensuring that the container is automatically restarted in case of failure.
- **ports**: Maps port 80 from the container to port 8889 on the host, allowing access to the pgAdmin web interface.
- **environment**: Defines the necessary environment variables to configure pgAdmin, including the default email and password.
- **network_mode**: Sets the network mode to "host" to simplify communication between containers.


#### app

The `app` service is configured using local image, it provides an API wich was required in future questions. Here's a explanation of the settings:

- **image/build**: Specifies the application image or the Dockerfile used to build the image.
- **ports**: Maps ports from the container to the host, allowing external access to the application.
- **depends_on**: Defines dependencies between services, in this case it depends on db.

##### Dockerfile

In [1]:
!cat Dockerfile

FROM python:3.11
ENV POETRY_VIRTUALENVS_CREATE=false

WORKDIR app/
COPY . .

RUN pip install poetry

RUN poetry config installer.max-workers 10
RUN poetry install --no-interaction --no-ansi

CMD ["bash", "run.sh"]


In [2]:
!cat run.sh

#!/bin/bash
alembic upgrade head
python cloudwalk/main.py


## Explain the relationship between the tables.


The relationship between the "Clients" table and the "Loans" table is a **one-to-many** relationship, where:

- One client can have multiple loans.
- Each loan is associated with only one client.

#### Clients Table

Each row in the "Clients" table represents a unique client of the bank.

| Column Name   | Data Type         | Description                                                        |
|---------------|-------------------|--------------------------------------------------------------------|
| user_id       | int64             | Unique identifier for each user                                    |
| created_at    | datetime64[ns]    | Date and time when the user was created                            |
| status        | string            | The status of the user (approved, denied)                          |
| batch         | int64             | The batch to which the user belongs                                |
| credit_limit  | int64             | The credit limit assigned to the user                              |
| interest_rate | int64             | The annual interest rate assigned to the user                      |
| denied_reason | string            | The reason for denial of the user                                  |
| denied_at     | datetime64[ns]    | The date and time when the user was denied                         |

#### Loans Table

Each row in the "Loans" table represents a loan taken out by a client.

| Column Name   | Data Type         | Description                                                        |
|---------------|-------------------|--------------------------------------------------------------------|
| user_id       | int64             | Unique identifier for each user                                    |
| loan_id       | int64             | Unique identifier for each loan                                    |
| created_at    | datetime64[ns]    | Date and time when the loan was created                            |
| due_at        | datetime64[ns]    | Date and time when the loan is due                                 |
| paid_at       | datetime64[ns]    | Date and time when the loan was paid                               |
| status        | string            | The status of the loan (paid, default, ongoing)                    |
| loan_amount   | float64           | The amount of the loan                                             |
| tax           | float64           | The tax on the loan                                                |
| due_amount    | float64           | The total amount due on the loan                                   |
| amount_paid   | float64           | The amount paid on the loan, until the current moment              |


# SQL and Data Viz

#### Helper functions

In [11]:
import pandas as pd
from sqlalchemy import create_engine

from cloudwalk.settings import Settings


def dataframe_from_sql(sql_query):
    """
    Executa um SQL query utilizando SQLAlchemy e retorna um DataFrame.

    Args:
        sql_query (str): O SQL query a ser executado.
        db_url (str): URL do banco de dados para estabelecer a conexão.

    Returns:
        pd.DataFrame: Pandas DataFrame contendo os resultados do query.
    """
    try:
        db_url = Settings().DATABASE_URL
        # Criar engine SQLAlchemy
        engine = create_engine(db_url)
        
        # Executar o SQL query e obter os resultados como DataFrame
        df = pd.read_sql_query(sql_query, engine)
        
        return df
    except Exception as e:
        print("Erro ao executar o SQL query:", e)
        return None

## QUESTION 1:

```Identify the best month in terms of loan issuance. What was the quantity and amount lent in each month?```

The idea here is to group the data by month and year, so that we can check their respective quantities and values. Then, we order them by quantity in descending order to see which month has the highest number of loans.

In [13]:

sql = """
SELECT  extract(month from created_at) as month,
		extract(year from created_at) as year,
		COUNT(*) as quantity,
		SUM(loan_amount) as value
FROM loans
GROUP BY 1, 2
ORDER BY 3 DESC
"""

df = dataframe_from_sql(sql)
df.head()


,month,year,quantity,value
0,12.0,2023.0,17351,442464966.0
1,1.0,2024.0,16123,409112591.0
2,11.0,2023.0,13269,330839275.0
3,10.0,2023.0,11593,293005656.0
4,9.0,2023.0,8976,229573371.0


```The best month in terms of loan issuance is december of 2023```

## QUESTION 2

```Determine which batch had the best overall adherence?```

The idea is to calculate the adherence rate (which i used as being the product of the user-approved rate and loans-paid rate) for different batches in the lending operation.

1. **UserAdherence Common Table Expression (CTE)**:
   - This part calculates the adherence rate for each batch based on the number of approved users compared to the total number of users in each batch.

2. **LoanAdherence Common Table Expression (CTE)**:
   - This part calculates the adherence rate for each batch based on the number of paid loans compared to the total number of loans (excluding defaulted loans) in each batch. It joins the `Loans` table with the `Clients` table to ensure that only loans from approved users are considered.

3. **Main Query**:
   - The main query selects the batch, user-approved rate, loans-paid rate, and calculates the adherence rate (which is the product of the user-approved rate and loans-paid rate) for each batch.
   - It then joins the results from the UserAdherence and LoanAdherence CTEs based on the batch.
   - Finally, it orders the results by the adherence rate in descending order to identify the batch with the highest adherence rate.


In [20]:
sql = """
WITH UserAdherence AS (
    SELECT 
        batch,
        SUM(CASE WHEN status = 'approved' THEN 1 ELSE 0 END) AS approved_users,
        COUNT(*) AS total_users,
		ROUND(SUM(CASE WHEN status = 'approved' THEN 1 ELSE 0 END)::numeric/COUNT(*)::numeric, 2) as user_approved_rate
    FROM 
        Clients
    GROUP BY 
        batch
),
LoanAdherence AS (
    SELECT 
        batch,
        SUM(CASE WHEN l.status = 'paid' THEN 1 ELSE 0 END) AS paid_loans,
        COUNT(*) AS total_loans,
		ROUND(SUM(CASE WHEN l.status != 'default' THEN 1 ELSE 0 END)::numeric/COUNT(*)::numeric, 2) as loans_paid_rate
    FROM 
        Loans l
    JOIN 
        Clients c ON l.user_id = c.user_id
    GROUP BY 
        batch
)

SELECT 	u.batch as batch,
		u.user_approved_rate,
		l.loans_paid_rate,
		u.user_approved_rate * l.loans_paid_rate as adherence_rate
FROM UserAdherence as u
INNER JOIN LoanAdherence as l
	ON l.batch = u.batch
ORDER BY 4 desc
"""

df = dataframe_from_sql(sql)
df.head()

,batch,user_approved_rate,loans_paid_rate,adherence_rate
0,4,0.84,0.92,0.7728
1,1,0.81,0.92,0.7452
2,3,0.79,0.92,0.7268
3,2,0.76,0.91,0.6916


```The batch wich had the best overall adherence was batch 0```

## Question 3 
```Do different interest rates lead to different loan outcomes in terms of default rate?```


The idea here is to group the data by interest_rate, so that we can check their respective average default_rate. Then, we can see if any batch has different default_rate.

In [19]:
sql = """
SELECT		interest_rate,
    		ROUND(SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END)::numeric / COUNT(*), 3) AS default_rate_percentage,
            COUNT(*) as qnt_loans
FROM	loans as l
	INNER JOIN 	clients as c
		ON c.user_id = l.user_id
GROUP BY    interest_rate;
"""
df = dataframe_from_sql(sql)
df.head()

,interest_rate,default_rate_percentage,qnt_loans
0,20,0.081,38020
1,30,0.082,37421
2,70,0.082,37201
3,90,0.083,38066


```Based on this data we cannot conclude that different interest rates result in divergent loan outcomes in terms of default rate.```

## Question 4 
```Rank the best 10 and 10 worst clients. Explain your methodology for constructing this ranking.```

The query aims to rank users in a lending operation based on a composite metric.

1. **Common Table Expressions (CTEs)**:

    - **loans_per_user CTE**:
        - This CTE calculates some metrics for each user, including total loan amount, defaulted amount, non-defaulted loan amount, and credit utilization ratio. It groups the data by user_id and orders the result by credit utilization ratio and loan amount in descending order.
        
    - **max_loans CTE**:
        - This CTE calculates the maximum loan amount across all users from the loans_per_user CTE.

    - **user_metrics CTE**:
        - This CTE computes additional metrics for each user, such as default ratio and credit ratio, based on the loan metrics calculated in the loans_per_user CTE.

2. **Main Query**:
    - This query selects the user_id and computes a composite ```user_metric``` using a weighted combination of the default ratio, credit ratio, and normalized loan amount. The weights assigned to each component are 35% for default ratio, 50% for credit ratio, and 15% for the normalized loan amount. It utilizes the user_metrics CTE and cross joins with the max_loans CTE to normalize the loan amount.


based on this ```user_metric``` we got the following 10 users as TOP10 best users

In [24]:
sql = """
WITH loans_per_user as(
SELECT 	c.user_id,
		SUM(loan_amount) as loan_amount,
		SUM(CASE 
				WHEN l.status = 'default' THEN  loan_amount
				ELSE 0
		   END) as amount_default,
		SUM(CASE 
				WHEN l.status <> 'default' THEN  loan_amount
				ELSE 0
		   END) as amount_loans,
		SUM(loan_amount) / SUM(credit_limit) AS credit_utilization_ratio
FROM clients as c
INNER JOIN loans as l
	ON	c.user_id = l.user_id
GROUP BY c.user_id
ORDER BY 4 DESC, 2 DESC
),
max_loans as (
SELECT MAX(loan_amount) as max_loans
FROM loans_per_user
),
user_metrics as (
SELECT 	user_id,
		(amount_loans/(amount_default + amount_loans)) as default_ratio,
		(CASE
		 WHEN credit_utilization_ratio < 0.1 THEN 10*(credit_utilization_ratio)
		 WHEN credit_utilization_ratio <= 0.3 AND credit_utilization_ratio >= 0.1 THEN 1
		 ELSE 1 - (credit_utilization_ratio - 0.3) / 0.7
		 END) as credit_ratio,
		credit_utilization_ratio,
		amount_loans,
		amount_default
FROM loans_per_user
)
SELECT 	user_id,
		(default_ratio*0.35 + credit_ratio*0.5 + (amount_loans/max_loans)*0.15)  as user_metric
FROM user_metrics
CROSS JOIN max_loans
ORDER BY 2 DESC
LIMIT 10
"""
df = dataframe_from_sql(sql)
df.head(10)

,user_id,user_metric
0,31439,0.897949
1,82847,0.896332
2,55685,0.896127
3,631,0.894711
4,50963,0.893276
5,25812,0.890690
6,28843,0.890621
7,6715,0.890437
8,44398,0.890307
9,39341,0.889802


based on user_metric we got the following 10 users as TOP10 worst users

In [27]:
sql = """
WITH loans_per_user as(
SELECT 	c.user_id,
		SUM(loan_amount) as loan_amount,
		SUM(CASE 
				WHEN l.status = 'default' THEN  loan_amount
				ELSE 0
		   END) as amount_default,
		SUM(CASE 
				WHEN l.status <> 'default' THEN  loan_amount
				ELSE 0
		   END) as amount_loans,
		SUM(loan_amount) / SUM(credit_limit) AS credit_utilization_ratio
FROM clients as c
INNER JOIN loans as l
	ON	c.user_id = l.user_id
GROUP BY c.user_id
ORDER BY 4 DESC, 2 DESC
),
max_loans as (
SELECT MAX(loan_amount) as max_loans
FROM loans_per_user
),
user_metrics as (
SELECT 	user_id,
		(amount_loans/(amount_default + amount_loans)) as default_ratio,
		(CASE
		 WHEN credit_utilization_ratio < 0.1 THEN 10*(credit_utilization_ratio)
		 WHEN credit_utilization_ratio <= 0.3 AND credit_utilization_ratio >= 0.1 THEN 1
		 ELSE 1 - (credit_utilization_ratio - 0.3) / 0.7
		 END) as credit_ratio,
		credit_utilization_ratio,
		amount_loans,
		amount_default
FROM loans_per_user
)
SELECT 	user_id,
		(default_ratio*0.35 + credit_ratio*0.5 + (amount_loans/max_loans)*0.15)  as user_metric
FROM user_metrics
CROSS JOIN max_loans
ORDER BY 2 ASC
LIMIT 10
"""
df = dataframe_from_sql(sql)
df.head(10)

,user_id,user_metric
0,18498,0.000150
1,6640,0.000185
2,21752,0.000280
3,78776,0.000292
4,28467,0.000302
5,46115,0.000344
6,17457,0.000401
7,77893,0.000627
8,56839,0.000707
9,76479,0.000778


## Question 5
```What is the default rate by month and by batch?```

The query aims to calculate the default rate for each month and year in the lending operation. The breakdown is as follows:

1. **SELECT clause**:
   - The SELECT clause extracts the month and year from the `created_at` column of the `loans` table.
   - It calculates the default rate by dividing the sum of defaulted loan amounts by the sum of total loan amounts for each month and year combination.

2. **FROM clause**:
   - The FROM clause specifies that the data is retrieved from the `loans` table.
   - It also joins the `loans` table with the `clients` table using an INNER JOIN operation based on the `user_id` column.

3. **GROUP BY clause**:
   - The GROUP BY clause groups the data by the extracted month and year from the `created_at` column.
   - This means that the aggregation function (SUM) will be applied to each unique combination of month and year.

4. **ORDER BY clause**:
   - The ORDER BY clause orders the result set first by year (`extract(year from l.created_at)`) and then by month (`extract(month from l.created_at)`).


In [31]:
sql = """
SELECT	extract(month from l.created_at) as month, 
		extract(year from l.created_at) as year,
		SUM(CASE 
			WHEN l.status = 'default' THEN  loan_amount
			ELSE 0
	   	END)/SUM(loan_amount) as default_rate
FROM loans as l
INNER JOIN clients as c
	ON	c.user_id = l.user_id
GROUP BY extract(month from l.created_at), extract(year from l.created_at)
ORDER BY 2, 1
"""
df = dataframe_from_sql(sql)
df.head(50)

,month,year,default_rate
0,1.0,2020.0,0.171505
1,2.0,2020.0,0.214756
2,3.0,2020.0,0.074838
3,4.0,2020.0,0.137965
4,5.0,2020.0,0.180306
5,6.0,2020.0,0.125940
6,7.0,2020.0,0.137125
7,8.0,2020.0,0.112355
8,9.0,2020.0,0.110123
9,10.0,2020.0,0.157892


The SQL query aims to calculate the default rate for each batch in the lending operation. Here's the breakdown:

1. **SELECT clause**:
   - The SELECT clause selects the batch from the `clients` table.
   - It calculates the default rate by dividing the sum of defaulted loan amounts by the sum of total loan amounts for each batch.

2. **FROM clause**:
   - The FROM clause specifies that the data is retrieved from the `loans` table.
   - It also joins the `loans` table with the `clients` table using an INNER JOIN operation based on the `user_id` column.

3. **GROUP BY clause**:
   - The GROUP BY clause groups the data by the batch from the `clients` table.
   - This means that the aggregation function (SUM) will be applied to each unique batch.

4. **ORDER BY clause**:
   - The ORDER BY clause orders the result set by the batch (`c.batch`) in ascending order.


In [33]:
sql = """
SELECT	c.batch,
		SUM(CASE 
			WHEN l.status = 'default' THEN  loan_amount
			ELSE 0
	   	END)/SUM(loan_amount) as default_rate
FROM loans as l
INNER JOIN clients as c
	ON	c.user_id = l.user_id
GROUP BY c.batch
ORDER BY 1
"""
df = dataframe_from_sql(sql)
df.head(50)

,batch,default_rate
0,1,0.080069
1,2,0.086028
2,3,0.079485
3,4,0.079119


## Question 6
```Assess the profitability of this operation. Provide an analysis of the operation's timeline.```

The idea is to calculate the profit by loan_amoun - amount_paid, but for ongoing loans this simple calculation is not correct, the way i managed to solve this problem is to calculate the profit of ongoing loans by using due_amount - laon_amount and correcting this value by avg total paid_rate (1 - default_rate)

```ongoing_profit = (due_amount - loan_amount)*(1-default_rate)```

```finished_profit = (paid_amount - loan_amount)```

```profit = finished_profit + ongoing_profit```

The SQL query aims to calculate the profit and ongoing income for each month and year in the lending operation. Here's the breakdown:

1. **Common Table Expressions (CTEs)**:

    - **default_rate_year CTE**:
        - This CTE calculates the default rate for each month and year before November 2023. It groups the data by year and month, and calculates the default rate as the sum of defaulted loan amounts divided by the sum of total loan amounts for each month and year combination.

    - **default_rate_avg CTE**:
        - This CTE calculates the average default rate from the default_rate_year CTE.

    - **loans_per_year CTE**:
        - This CTE calculates various metrics for each month and year, including the total amount paid, defaulted amount, and ongoing income. It utilizes the default_rate_avg CTE to calculate the ongoing income based on the default rate average. The data is grouped by year and month.

2. **Main Query**:
    - This query selects the year, month, profit (sum of paid and defaulted amounts), and ongoing income for each month and year.
    - It orders the result set by year.


In [35]:
sql = """
WITH default_rate_year as ( 
SELECT  extract(month from created_at),
		extract(year from created_at),
		SUM(CASE WHEN status='default' THEN loan_amount ELSE 0 END)/SUM(loan_amount) as default_rate
FROM loans
WHERE created_at::date < '2023-11-01'::date
GROUP BY extract(year from created_at), extract(month from created_at)
ORDER BY 2,1
),
default_rate_avg as (
SELECT AVG(default_rate) default_rate_avg
FROM default_rate_year
),
loans_per_year as (
SELECT 		extract(month from created_at) as month,
			extract(year from created_at) as year,
			SUM(CASE
		   		WHEN status = 'paid' THEN amount_paid - loan_amount
		   		ELSE 0
		  	END)::int as paid,
			SUM(CASE
		   		WHEN status = 'default' THEN amount_paid - loan_amount
		   		ELSE 0
		  	END)::int as "default",
			SUM(CASE
		   		WHEN status = 'ongoing' THEN (due_amount - loan_amount)*default_rate_avg
		   		ELSE 0
		  	END)::int as "ongoing"
			
FROM loans
CROSS JOIN default_rate_avg
GROUP BY extract(year from created_at),  extract(month from created_at)
)
SELECT 	"year",
		"month",
		paid + "default" as finished_profit,
		ongoing as ongoing,
        paid + "default" + ongoing as profit
FROM loans_per_year
ORDER BY year
"""
df = dataframe_from_sql(sql)
df.head(50)

,year,month,finished_profit,ongoing,profit
0,2020.0,1.0,-802,0,-802
1,2020.0,2.0,-13389,0,-13389
2,2020.0,3.0,305405,0,305405
3,2020.0,4.0,293047,0,293047
4,2020.0,5.0,237439,0,237439
5,2020.0,6.0,276661,0,276661
6,2020.0,7.0,440526,0,440526
7,2020.0,8.0,628711,0,628711
8,2020.0,9.0,666473,0,666473
9,2020.0,10.0,706455,0,706455


# Infra  & Python

## Question 1
```Develop functions to add and alter users in the clients table in a safe way.```

```The idea here is to create an API to interact with database and add, alter or delete users in the clients table in a safe way```

#### FastAPI


FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints. It is designed to be easy to use, fast to develop with, and highly efficient. FastAPI leverages Python's asynchronous capabilities to provide high-performance web applications and APIs.

Key features of FastAPI include:

- **Automatic API Documentation**: FastAPI automatically generates interactive API documentation (Swagger UI and ReDoc) based on your code's type hints, making it easy to understand and test your API endpoints.

- **Type Checking**: FastAPI uses Python type hints to perform runtime type checking and validation, ensuring that data passed to your API endpoints is of the correct type and structure.

- **Async Support**: FastAPI fully supports asynchronous programming with Python's `asyncio` library, allowing you to write non-blocking, asynchronous code that can handle high concurrency and IO-bound operations efficiently.

- **Dependency Injection**: FastAPI provides built-in support for dependency injection, allowing you to easily inject dependencies into your endpoint functions and organize your code in a clean and modular way.

- **Performance**: FastAPI is built on top of Starlette and Pydantic, two highly optimized libraries, resulting in excellent performance and low overhead.

- **Easy Integration**: FastAPI seamlessly integrates with popular Python libraries and frameworks, including SQLAlchemy, Tortoise-ORM, and more, making it easy to incorporate into your existing projects.

Get started with FastAPI today and build high-performance web APIs with ease!

In [36]:
!curl  http://localhost:8000/ping

{"message":"pong"}

In [41]:
!curl -X POST http://localhost:8000/users/ \
     -H "Content-Type: application/json" \
     -d '{\
        "status": "denied",\
        "batch": 0,\
        "credit_limit": 0,\
        "interest_rate": 0,\
        "denied_reason": "money_loundry",\
        "denied_at": "2024-02-17T17:22:48.988Z"\
     }'


{"user_id":90001,"status":"denied"}

In [43]:
!curl -X PUT http://localhost:8000/users/90001 \
     -H "Content-Type: application/json" \
     -d '{\
        "status": "approved",\
        "batch": 0,\
        "credit_limit": 10000,\
        "interest_rate": 20\
     }'

{"user_id":90001,"status":"approved"}

In [44]:
!curl -X DELETE http://localhost:8000/users/90001 

{"message":"user deleted"}

## Question 2
```Propose processes that should run daily, weekly, or monthly to maintain a healthy operation.```

## Question 3
```Develop an automated email service to remind users with ongoing loans about payments. Select the frequency and content as you see fit.```

### rocketry

#### Introduction to Rocketry Scheduler

Rocketry Scheduler is a lightweight Python library designed to help you schedule and automate tasks within your Python applications effortlessly. With Rocketry Scheduler, you can define tasks to be executed at specific times or intervals, making it easy to automate routine processes, periodic updates, and more.

Key features of Rocketry Scheduler include:

- **Simple Task Scheduling**: Define tasks using plain Python functions or methods and schedule them to run at specified times or intervals.
  
- **Flexibility**: Tasks can be one-time events or recurring jobs, providing flexibility in task execution.

- **Easy Integration**: Integrate Rocketry Scheduler seamlessly into your Python applications with minimal effort.

- **Error Handling**: Handle exceptions and failures gracefully with built-in error handling mechanisms.

- **Lightweight and Efficient**: Rocketry Scheduler is lightweight and resource-efficient, making it suitable for a wide range of applications.


#### Email Handler

The idea is to utilize aiosmtplib to create async email sender, then uses a default email remember users with ongoing loans that they have to pay.

```
from email import message
import aiosmtplib
from sqlalchemy import text

from cloudwalk.db.engine import get_session
from cloudwalk.settings import Settings


class SmtpHandler:
    def __init__(self, settings: Settings):
        self.smpt_url = settings.SMTP_URL
        self.smpt_port = 465
        self.smpt_user = settings.EMAIL_SENDER
        self.smpt_password = settings.EMAIL_PASSWORD

    async def sendmail(self, body: str, subject: str, email_receiver: list):
        em = message.EmailMessage()
        em['From'] = self.smpt_user
        em['Subject'] = subject
        em.set_content(body)
        smtp = aiosmtplib.SMTP(
            hostname=self.smpt_url, port=self.smpt_port, use_tls=True
        )
        await smtp.connect()
        await smtp.login(self.smpt_user, self.smpt_password)
        await smtp.sendmail(self.smpt_user, email_receiver, em.as_string())
        await smtp.quit()
```

Then create OnGoiongLoansEmail wich uses SmtpHandler to send a deault email to  users with ```status = ongoing``` and ```created_date % 30 = 0```

```
class OnGoingLoansEmail:
    def __init__(self, smtp: SmtpHandler):
        self.smtp = smtp

    def _get_users(self):
        session = get_session()
        sql = text(
            """
            SELECT DISTINCT user_id
            FROM loans
            WHERE 		status = 'ongoing'
                AND 	extract(day from (CURRENT_DATE - created_at))%30 = 0
        """
        )
        results = [
            str(item[0]) + '@cloudwalk.com' for item in session.execute(sql)
        ]
        return results[:10]

    async def run(self):
        users = self._get_users()
        subject = 'Lembrete de Pagamento de Empréstimo'
        body = """
        Caro Cliente,
        Esperamos que este email o encontre bem.
        Gostaríamos de lembrar a todos os nossos valiosos clientes que é fundamental manter-se em dia com os pagamentos de empréstimos. Entendemos que a gestão financeira pode ser desafiadora, e estamos aqui para ajudar a tornar o processo o mais simples possível.
        Por favor, verifique a data de vencimento do seu empréstimo e assegure-se de que o pagamento seja efetuado até essa data. Manter-se em dia com os pagamentos é essencial para evitar quaisquer encargos adicionais ou penalidades.
        Se você já efetuou o pagamento, agradecemos sinceramente sua cooperação. Caso contrário, pedimos que tome as medidas necessárias para garantir que o pagamento seja realizado a tempo.
        Lembre-se de que estamos aqui para ajudar. Se precisar de qualquer assistência adicional ou tiver alguma dúvida sobre o seu empréstimo, não hesite em nos contatar. Estamos disponíveis para ajudá-lo da melhor forma possível.
        Agradecemos pela sua atenção e cooperação contínuas.
        """
        await self.smtp.sendmail(body, subject, users)
```

## Question 4
```Create an automated weekly email summarizing operation activities. Define the layout and information included. Submission```

In [50]:
sql = """
WITH default_rate_year as ( 
SELECT  extract(month from created_at),
        extract(year from created_at),
        SUM(CASE WHEN status='default' THEN loan_amount ELSE 0 END)/SUM(loan_amount) as default_rate
FROM loans
WHERE created_at::date < '2023-11-01'::date
GROUP BY extract(year from created_at), extract(month from created_at)
ORDER BY 2,1
),
default_rate_avg as (
SELECT AVG(default_rate) default_rate_avg
FROM default_rate_year
)

SELECT 
    EXTRACT(WEEK FROM l.created_at) AS week,
    EXTRACT(YEAR FROM l.created_at) AS year,
    COUNT(*) AS total_loans,
    SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END) AS defaulted_loans,
    SUM(CASE WHEN l.status = 'ongoing' THEN 1 ELSE 0 END) AS ongoing_loans,
    ROUND(SUM(l.due_amount)::numeric, 2) AS total_revenue,
    ROUND(SUM(l.loan_amount)::numeric, 2) AS total_loan_amount,
    ROUND(SUM(l.amount_paid - l.loan_amount)::numeric, 2) AS profit,
    ROUND(
        (
            SUM(CASE 
                    WHEN l.status = 'ongoing' THEN (l.amount_paid - l.loan_amount) + (l.due_amount - l.amount_paid)*(1 - d.default_rate_avg)
                    ELSE 0
                END)
        )::numeric, 2
    ) AS on_going_profit	
FROM	loans as l CROSS JOIN default_rate_avg as d
WHERE 
    EXTRACT(WEEK FROM created_at) = EXTRACT(WEEK FROM CURRENT_DATE) - 4
    AND EXTRACT(YEAR FROM created_at) = EXTRACT(YEAR FROM CURRENT_DATE)
GROUP BY 
    EXTRACT(WEEK FROM created_at), EXTRACT(YEAR FROM created_at);
"""
df = dataframe_from_sql(sql)
df.head()

,week,year,total_loans,defaulted_loans,ongoing_loans,total_revenue,total_loan_amount,profit,on_going_profit
0,3.0,2024.0,4942,0,4942,1.436443e+08,125831087.0,-53527569.7,8902557.62


```The idea here is to send this informations on a email as an automated weekly email summarizing operation activities```